In [ ]:
%load_ext autoreload
%autoreload 2

import lib.TimeKeeper as tk
import lib.Toolbox as tb
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tb.get_archive('CVNA')
bollinger = tb.get_bollinger_bands(data.open, num_stds=(1.5,0,-1.5))

In [ ]:
success, total = 0, 0
earnings = 0
points = []
for index in data.index[1:-5]:
    pindex = tb.get_previous_index(index, data)
    val, pval =  data.open.loc[index], data.open.loc[pindex]
    bol_high, pbol_high = bollinger.highband.loc[index], bollinger.highband.loc[pindex]
    bol_low, pbol_low = bollinger.lowband.loc[index], bollinger.lowband.loc[pindex]
    
    # Shortsell
    if val < bol_high and pval > pbol_high:
        window = data.open[index+pd.DateOffset(minutes=1):index + pd.DateOffset(minutes=15)]
        total += 1
        if val > window.min() and window.min() > val - .03:
            points.append([])
            success += 1
            #earnings += (val - window.min()) * 1000
            earnings += .03 * 1000
        else:
            change = (data.open.] - val) * 1000
            earnings += change

    # Buy
    elif val > bol_low and pval < pbol_low:
        window = data.open[index+pd.DateOffset(minutes=1):index + pd.DateOffset(minutes=15)]
        total += 1
        if val < window.max() and  window.max() > val + .03:
            success += 1
            #earnings += (window.max() - val) * 1000
            earnings += .03 * 1000
        else:
            change = (val - data.open.loc[index+pd.DateOffset(minutes=15)]) * 1000
            earnings += change
            

In [ ]:
print(success, total)
print(success/total)
print(earnings)

# Archive Collection
Manually identify some stocks and then create archives for them (as well as a function to update the archives of all important stocks on a daily basis. We'll put this in tb when it's done and then run it at the start of each day)


# Create Bollinger Effectiveness Rating
The BER is a personal technical indicator which determines whether the give stock is effective for the bollinger bands technical indicator. Basically, it asks how many places that the bollinger bands are triggered are then followed by some outcome. A relative BER indicates asks what % of bollinger events then lead to a paired bollinger event, and an absolute BER asks how many events are followed by a profit of .05 cents (or some other value).

In [ ]:
PROFIT_MOD = 2000

In [ ]:
class TradeEvent:
    def __init__(self, index:pd.DatetimeIndex, type:int, data:pd.DataFrame, end:pd.DatetimeIndex):
        self.index, self.type, self.data, self.end = index, type, data, end
        self.value = self.data.loc[self.index].open

    def set_window(self, cap:int):
        self.window = data[self.index+pd.DateOffset(minutes=1):self.index+pd.DateOffset(minutes=cap)]
        if len(self.window) > 0:
            self.winmax, self.winmin = self.window.loc[self.window.idxmax().open].open, self.window.loc[self.window.idxmin().open].open
            self.cutoff_value = self.window.iloc[-1].open
            self.best_possible_profit = (self.winmax - self.value) * PROFIT_MOD if self.type == 1 else (self.value - self.winmin) * PROFIT_MOD
            return True
        return False

    def evaluate_event(self, modifier:float):
        '''Evaluates whether the given event is a success or failure and determines the profit that we would get from it'''
        if self.type == 1:
            if (self.value + modifier) < self.winmax:
                self.profit = modifier * PROFIT_MOD
                return True
            else:
                self.profit = (self.cutoff_value - self.value) * PROFIT_MOD
                return False
        
        elif self.type == -1:
            if (self.value - modifier) > self.winmin: 
                self.profit = modifier * PROFIT_MOD
                return True
            else:
                self.profit = (self.value - self.cutoff_value) * PROFIT_MOD
                return False

    def show(self):
        '''Displays the given event'''
        plt.title(self.index)
        canvas = self.data[self.index - pd.DateOffset(minutes=len(self.window)):self.index + pd.DateOffset(minutes=len(self.window)*2)]
        plt.plot(canvas.index, canvas.open, lw=.5, color='blue')
        if self.type == 1: plt.axvline(self.index, lw=.5, color='green')
        elif self.type == -1: plt.axvline(self.index, lw=.5, color='red')
        plt.plot(self.window.index, self.window.open, lw=.75, color='purple')
        plt.show()

def check_bollinger(data:pd.DataFrame, index:pd.DatetimeIndex, bollinger:pd.DataFrame):
    pindex = data.index[index-1]
    index = data.index[index]
    val, pval, bol, pbol = data.open.loc[index], data.open.loc[pindex], bollinger.loc[index], bollinger.loc[pindex]
    if val > bol.lowband and pval < pbol.lowband: return 1
    elif val < bol.highband and pval > pbol.highband: return -1
    return 0

def find_bollinger_events(data:pd.DataFrame, bol_mod:int):
    '''Runs through the data and finds bollinger events (separated by day)'''
    event_list = []
    for day in tb.get_unique_dates(data):
        dayta = tk.get_workday(data, day)
        bollinger = tb.get_bollinger_bands(dayta.open, num_stds=(bol_mod, 0, -bol_mod))
        for i in range(len(dayta.index))[1:]:
            bol_key = check_bollinger(dayta, i, bollinger)
            if bol_key == 1: event_list.append(TradeEvent(dayta.index[i], 1, data, dayta.index[-1]))
            elif bol_key == -1: event_list.append(TradeEvent(dayta.index[i], -1, data, dayta.index[-1]))
    return event_list

def evaluate_absolute_bollinger(modifier:float, events:list, offset:int):
    '''Determines the percentage of bollinger events that have a profit of the modifier with the time of the offset'''
    successful_events, failed_events = [], []
    for event in events:
        if not event.set_window(offset): continue
        if event.evaluate_event(modifier): successful_events.append(event)
        else: failed_events.append(event)
       
    return len(successful_events)/(len(successful_events) + len(failed_events)), successful_events, failed_events

def evaluate_relative_bollinger(data:pd.DataFrame, bollinger:pd.DataFrame, events:list):
    '''Determines the percentage of bollinger events that have a matching pair later in the day'''
    successful_events, failed_events = [], []
    for event in events:
        if search_for_pair(data[event.index:event.end], bollinger, event): successful_events.append(event)
        else: failed_events.append(event)
    return len(successful_events)/(len(successful_events) + len(failed_events)), successful_events, failed_events

def search_for_pair(data:pd.DataFrame, bollinger:pd.DataFrame, event:TradeEvent):
    for i in range(len(data.index))[1:]:
        index = data.index[i]
        if check_bollinger(data, i, bollinger) != event.type:
            bol_val = data.open.loc[index]
            if event.type == 1:
                if bol_val > event.value:
                    event.profit = (bol_val-event.value)*PROFIT_MOD
                    return True
            elif event.type == -1:
                if bol_val < event.value:
                    event.profit = (event.value-bol_val)*PROFIT_MOD
                    return True
    if event.type == 1: event.profit = (data.open.loc[event.end]-event.value)*PROFIT_MOD
    elif event.type == -1: event.profit = (event.value-data.open.loc[event.end])*PROFIT_MOD
    return False

In [ ]:
data = tb.get_archive('CVNA')

# GENERATE WINS AND LOSSES

In [ ]:
total_profit = 0
# profit analysis
bollinger = tb.get_bollinger_bands(data.open, num_stds=(1.5,0,-1.5))
rate, wins, losses = evaluate_relative_bollinger(data, bollinger, find_bollinger_events(data, 1.5))
print(rate, len(wins), len(losses))

for event in wins: total_profit += event.profit
#for event in wins: total_profit += event.best_possible_profit
for event in losses: total_profit += event.profit
print(total_profit, total_profit/len(tb.get_unique_dates(data)))

In [ ]:
count, l = 0, wins
for trade in l:
    noon = tk.get_midnight(trade.index)+pd.DateOffset(hours=14)
    if trade.index > noon: count += 1
print(count/len(l))

In [ ]:
# Rapid Acceleration Testing
def find_acceleration_events(data:pd.DataFrame):
    '''Runs through the data and finds acceleration events (separated by day)'''
    event_list = []
    for day in tb.get_unique_dates(data):
        dayta = tk.get_workday(data, day)
        acceleration = dayta.open.ewm(span=30).mean().diff().ewm(span=30).mean().diff()
        for index in range(len(dayta.index))[1:]:
            pindex = dayta.index[index-1]
            index = dayta.index[index]
            acc, pacc = acceleration.loc[index], acceleration.loc[pindex]
            if acc > 0 and pacc < 0: event_list.append(TradeEvent(index, 1, data))
            if acc < 0 and pacc > 0: event_list.append(TradeEvent(index, -1, data))
    return event_list

def evaluate_absolute_acceleration(modifier:float, events:list, offset:int):
    '''Determines the percentage of acceleration events that have a profit of the modifier with the time of the offset'''
    successful_events, failed_events = [], []
    for event in events:
        if not event.set_window(offset): continue
        if event.evaluate_event(modifier): successful_events.append(event)
        else: failed_events.append(event)
       
    return len(successful_events)/(len(successful_events) + len(failed_events)), successful_events, failed_events

In [ ]:
rate, wins, losses = evaluate_absolute_acceleration(.02, find_acceleration_events(data), 15)
print(rate, len(wins), len(losses))

In [ ]:
# Analyze the Failure States
data = tb.get_archive('CVNA')
rate, wins, losses = evaluate_absolute_bollinger(.05, find_bollinger_events(data, 1.5), 60)
print(rate, len(wins), len(losses))

In [ ]:
total_profit = 0
# profit analysis

for event in wins: total_profit += event.profit
#for event in wins: total_profit += event.best_possible_profit
for event in losses: total_profit += event.profit
print(total_profit, total_profit/len(tb.get_unique_dates(data)))

# MACD ANALYSIS

In [ ]:
upwin, downwin, uploss, downloss = [], [], [], []
macd = tb.get_macd(data, 12, 26, 9)

def get_macd(event, macd):
    return macd.macd_h.loc[event.index]

for event in wins:
    t = get_macd(event, macd)
    if event.type == 1: upwin.append(t)
    else: downwin.append(t)
for event in losses:
    t = get_macd(event, macd)
    if event.type == 1: uploss.append(t)
    else: downloss.append(t)
upwin, downwin, uploss, downloss = np.array(upwin), np.array(downwin), np.array(uploss), np.array(downloss)

uploss_threshold = uploss.mean() + uploss.std()*1.5
downloss_threshold = downloss.mean() - downloss.std()*1.5
print('uploss mean',uploss.mean(), uploss.std())
print('downloss mean',downloss.mean(), downloss.std())
print(uploss_threshold, downloss_threshold)

total_profit, w, l = 0, 0, 0
for event in wins:
    t = get_macd(event, macd)
    if event.type == 1 and t > uploss_threshold: w += event.profit
    elif event.type == -1 and t < downloss_threshold: w += event.profit
for event in losses:
    t = get_macd(event, macd)
    if event.type == 1 and t > uploss_threshold: l += event.profit
    elif event.type == -1 and t < downloss_threshold: l += event.profit
total_profit = w+l
print(round(total_profit,2), round(total_profit/len(tb.get_unique_dates(data)),2), '\n\tWins:',round(w/len(tb.get_unique_dates(data)),2), '\n\tLosses:',round(l/len(tb.get_unique_dates(data)),2),'\n\t', round(abs(w/l),2))

# TREND ANALYSIS

In [ ]:
def trend_analysis(event:TradeEvent, data:pd.DataFrame):
    canvas = data[event.index - pd.DateOffset(minutes=15):event.index]
    return tb.get_trend(canvas.open)

upwin, downwin, uploss, downloss = [], [], [], []
for event in wins:
    t = trend_analysis(event, data)
    if event.type == 1: upwin.append(t)
    else: downwin.append(t)
for event in losses:
    t = trend_analysis(event, data)
    if event.type == 1: uploss.append(t)
    else: downloss.append(t)
upwin, downwin, uploss, downloss = np.array(upwin), np.array(downwin), np.array(uploss), np.array(downloss)

uploss_threshold = uploss.mean()# + uploss.std()
downloss_threshold = downloss.mean()# - downloss.std()
print('uploss mean',uploss.mean(), uploss.std())
print('downloss mean',downloss.mean(), downloss.std())
print(uploss_threshold, downloss_threshold)

total_profit, w, l = 0, 0, 0
for event in wins:
    if event.type == 1 and trend_analysis(event, data) > uploss_threshold: w += event.profit
    elif event.type == -1 and trend_analysis(event, data) < downloss_threshold: w += event.profit
for event in losses:
    if event.type == 1 and trend_analysis(event, data) > uploss_threshold: l += event.profit
    elif event.type == -1 and trend_analysis(event, data) < downloss_threshold: l += event.profit
    total_profit = w+l
print(round(total_profit,2), round(total_profit/len(tb.get_unique_dates(data)),2), '\n\tWins:',round(w/len(tb.get_unique_dates(data)),2), '\n\tLosses:',round(l/len(tb.get_unique_dates(data)),2),'\n\t', round(abs(w/l),2))

# RSI ANALYSIS

In [ ]:
upwin, downwin, uploss, downloss = [], [], [], []
rsi = tb.get_rsi(data, 14)

def get_rsi(event, rsi):
    return rsi.loc[event.index]

for event in wins:
    t = get_rsi(event, rsi)
    if event.type == 1: upwin.append(t)
    else: downwin.append(t)
for event in losses:
    t = get_rsi(event, rsi)
    if event.type == 1: uploss.append(t)
    else: downloss.append(t)
upwin, downwin, uploss, downloss = np.array(upwin), np.array(downwin), np.array(uploss), np.array(downloss)

uploss_threshold = uploss.mean() + uploss.std()*1.5
downloss_threshold = downloss.mean() - downloss.std()*1.5
print('uploss mean',uploss.mean(), uploss.std())
print('downloss mean',downloss.mean(), downloss.std())
print(uploss_threshold, downloss_threshold)

total_profit, w, l = 0, 0, 0
for event in wins:
    t = get_rsi(event, rsi)
    if event.type == 1 and t > uploss_threshold: w += event.profit
    elif event.type == -1 and t < downloss_threshold: w += event.profit
for event in losses:
    t = get_rsi(event, rsi)
    if event.type == 1 and t > uploss_threshold: l += event.profit
    elif event.type == -1 and t < downloss_threshold: l += event.profit
total_profit = w+l
print(round(total_profit,2), round(total_profit/len(tb.get_unique_dates(data)),2), '\n\tWins:',round(w/len(tb.get_unique_dates(data)),2), '\n\tLosses:',round(l/len(tb.get_unique_dates(data)),2),'\n\t', round(abs(w/l),2))

In [ ]:
# Find Success Rates for Our Tracked Stocks
for symbol in tb.get_symbols('tracked_tickers.txt'):
    print(symbol)
    for mod in [1, 1.5, 2]:
        data = tb.get_archive(symbol)
        rate, mod = evaluate_absolute_bollinger(data, .05, find_bollinger_events(data, mod), 60), str(mod).rjust(3,'0')
        print(f'\tFor bol_mod {mod}: {round(rate,3)}')


# TREND SELECTION (AGAIN)

In [ ]:
data = tb.get_archive('CVNA')

In [ ]:
# DISPLAY HOUR TREND
unit = tb.to_unit(df_hour)
plt.figure(dpi=300)
plt.plot(unit.index, unit.open, lw=.5)
bollinger = tb.get_bollinger_bands(unit.open, 10, num_stds=(2, 0, -2))
plt.plot(bollinger.index, bollinger.highband, lw=.25, color='red')
plt.plot(bollinger.index, bollinger.moving_average, lw=.25, color='orange')
plt.plot(bollinger.index, bollinger.lowband, lw=.25, color='green')
for i in range(len(df_hour.index)):
    if df_hour.index[i].hour == 10:
        plt.axvline(i, color='violet', lw=.25)

# Predictions
Let's create a new kind of way of looking at things, i.e, let's create a new system of trade signals which will determine whether or not we expect the next span of time will grow or decline.

In [ ]:
class TradeSignal:
    '''A class meant to monitor large scale trade signal changes to determine whether the immediate future will be characterized by growth, or decline'''
    

In [ ]:
#days = tb.get_unique_dates(df_hour)
# for day in days:
#     dayta = tk.get_workday(data, day)
#     for index in dayta.index:
#         print(tb.get_index(index, df_hour))
df_hour = tb.fractionate_data(data, 30)
bollinger_hour = tb.get_bollinger_bands(df_hour.open, 20, num_stds=(2,0,-2))
for hour in df_hour.index:
    signal = tb.check_bollinger(hour, df_hour, bollinger_hour)
    if signal == 1:
        window = data[hour:hour+pd.DateOffset(hours=8)]
        plt.title(hour)
        plt.plot(window.index, window.open, color='green', lw=.5)
        plt.show()
    elif signal == -1:
        window = data[hour:hour+pd.DateOffset(hours=8)]
        plt.title(hour)
        plt.plot(window.index, window.open, color='red', lw=.5)
        plt.show()

# MARKET ANALYSIS
Moving averages

200 day average
If it's horrizontal than the price is moving
buy signal is 50 day moving average crossing above 200 day moving average and vice versa

If the MACD is above zero then the 

Below 30 RSI means it's oversold, 70 means overbought

OBV takes volume information
a rising obv should go with a rising price, obv confirms trends.

In [ ]:
macd = tb.get_macd(data, 50, 200, 50)
tb.to_unit(macd).macd_h.plot.line()

# Weirdness Calculator

In [ ]:
days = tb.get_unique_dates(data)

In [ ]:
weird_days = [[9, 11],
[9, 14],
[9, 15],
[9, 19],
[9, 20],
[9, 21],
[9, 28],
[10, 2],
[10, 6],
[10, 9],
[10, 11],
[10, 16],
[10, 18],
[10, 23],
[11, 7],
[11, 9],
[11, 21],
[11, 29],
[12, 1],
[12, 6]]

In [ ]:
def get_weirdness_score(index:pd.DatetimeIndex, data:pd.DataFrame):
    '''Attempts to measure the weirdness and potential out of controlness of a given day'''
    delta = data[:index].open.diff()
    pos, neg = delta[delta > 0], delta[delta < 0]
    weirdness_score = (pos.mean()/abs(neg.mean()))
    if not pd.isna(weirdness_score): return weirdness_score
    return 0

for day in weird_days:
    day = tk.to_time(day[0],day[1])
    dayta = tk.get_workday(data, day)
    weirdness_total = 0
    for index in dayta.index:
        weirdness_total += get_weirdness_score(index, dayta)
        weirdness_total /= 2
    print(weirdness_total)
    dayta.open.plot.line()
    plt.title(day)
    plt.show()

In [ ]:
for day in days:
    dayta = tk.get_workday(data, day)
    weirdness_total = 0
    for index in dayta.index:
        weirdness_total += get_weirdness_score(index, dayta)
        weirdness_total /= 2
    print(weirdness_total)
    dayta.open.plot.line()
    plt.title(day)
    plt.show()

# for day in days:
#     dayta = tk.get_workday(data, day)
#     dayta.open.plot.line()
#     plt.title(day)
#     plt.show()

# Pattern Rec Attempt Two

In [ ]:
data = tb.get_data('CVNA', '5m','30d')
seed = data[tk.to_time(12,29):tk.to_time(12,30)]
p = Pattern(seed, data)

In [ ]:
seed.open.plot.line()

In [ ]:
trend = round(tb.get_trend(seed.open),2)
for day in tb.get_unique_dates(data):
    dayta = tk.get_workday(data, day)
    day_trend = round(tb.get_trend(dayta[:27].open),2)
    if day_trend -.2 < trend and day_trend + .2 > trend:
        dayta[:27].open.plot.line()
        dayta[26:].open.plot.line()
        plt.title(day)
        plt.show()


In [ ]:
class Pattern:
    def __init__(self, seed:pd.DataFrame, data:pd.DataFrame):
        self.seed, self.data = seed, data
        self.pattern = self.process_seed(seed)
        self.recognize_pattern(self.pattern, data)
    
    def process_seed(self, seed:pd.DataFrame):
        '''Processes the seed into a normalized, smoothed pattern line'''
        pattern = seed.open.ewm(span=5).mean(); pattern = tb.normalize_column(pattern, True)
        return pattern

    def recognize_pattern(self, pattern:pd.DataFrame, data:pd.DataFrame):
        '''Trawls through the data looking for a pattern that matches the seed'''
        forecast = 0
        for i in range(0, len(data.index)-len(pattern)*2, 30):
            if data.index[i] in self.seed.index: continue
            segment = self.process_seed(data.iloc[i:i+len(pattern)])
            differential = abs(pattern - segment).sum()/len(pattern)
            if differential < .30:
                prediction = data.iloc[i+len(pattern):i+len(pattern)*2]
                upred = tb.normalize_column(prediction, True)
                forecast += tb.get_trend(upred.open)
        self.forecast = forecast

    def plot_pattern(self):
        '''Plots the pattern and any recognized patterns on the initial data'''

In [ ]:
data = tb.get_data('CVNA', '5m','60d')

In [ ]:
total, good = 0, 0
for date in tb.get_unique_dates(data)[6:]:
    seed = data[date-pd.DateOffset(hours=18):date]
    if len(seed) == 0: seed = data[date-pd.DateOffset(days=3):date]
    p = Pattern(seed, data[:date])
    day_performance = tb.get_trend(data[date:date+pd.DateOffset(days=1)].open)
    total += 1
    if p.forecast < 0 and day_performance < 0: good += 1
    elif p.forecast > 0 and day_performance > 0: good += 1
    print(date, p.forecast, day_performance)
print(good/total)

In [ ]:
seed.open.plot.line()

In [ ]:
data = tb.get_data('CVNA', '1d', '6mo')

In [ ]:
lookback = 1

In [ ]:

#plt.figure(dpi=300)
#plt.plot(delta.index, delta, lw=.5)
#plt.axhline(0, color='red',lw=.5)
def get_delta(data, ser, date):
    delta = ser - data.open
    d_i = tb.get_i(date, data)
    subdelta = delta.iloc[d_i-lookback:d_i]
    subdiff = subdelta.diff().mean()
    return subdelta.mean() + subdiff

In [ ]:
total_diff = 0
for date in data.index[lookback:]:
    plow = get_delta(data, data.low, date)
    phigh = get_delta(data, data.high, date)
    day = data.loc[date]
    print(date)
    print('\tLOW PREDICTION:',round(day.low,2), '\t',round(day.open + plow,2),'\t', round(day.low - (day.open + plow),2))
    print('\tHIGH PREDICTION:',round(day.high,2), '\t',round(day.open + phigh,2),'\t', round(day.high - (day.open + phigh),2))
    total_diff += abs(day.low - (day.open + plow)) + abs(day.high - (day.open + phigh))
print(total_diff/len(data.index[lookback:]))

# Multi Bollinger Test

In [ ]:
hdata = tb.get_data('CVNA','1h','7d')
tdata = tb.get_data('CVNA','30m','7d')
fdata = tb.get_data('CVNA','5m','7d')
data = tb.get_archive('CVNA')[fdata.index[0]:]
# hdata = data.ewm(span=60).mean()
# tdata = data.ewm(span=30).mean()
# fdata = data.ewm(span=5).mean()

bol_dev = 1

hbol = tb.get_bollinger_bands(hdata.open, num_stds=(bol_dev,0,-bol_dev))
tbol = tb.get_bollinger_bands(tdata.open, num_stds=(bol_dev,0,-bol_dev))
fbol = tb.get_bollinger_bands(fdata.open, num_stds=(bol_dev,0,-bol_dev))
bol = tb.get_bollinger_bands(data.open, num_stds=(bol_dev,0,-bol_dev))

In [ ]:
switchbox = [0,0,0]
signal_list = []
# Search Hours for Bollinger Signal
start_hour = 0
for hi in range(len(hdata[:tk.to_time(12,29)])):
    hour_sig = tb.check_bollinger(hi, hdata, hbol)
    if hour_sig != 0: start_hour = hi; break
if start_hour == 0:
    # Look back farther in history
    print('Hour signal not found')
for i in range(len(data[:tk.to_time(12,29)]))[start_hour*60:]:
    hi, ti, fi = int(i/60), int(i/30), int(i/5)

    hour_sig = tb.check_bollinger(hi, hdata, hbol)
    thirt_sig = tb.check_bollinger(ti, tdata, tbol)
    five_sig = tb.check_bollinger(fi, fdata, fbol)

    if hour_sig == 1: switchbox[0] = 1
    elif hour_sig == -1: switchbox[0] = -1
    if five_sig == 1: switchbox[1] = 1
    elif five_sig == -1: switchbox[1] = -1
    if thirt_sig == 1: switchbox[2] = 1
    elif thirt_sig == -1: switchbox[2] = -1
    
    signal = switchbox[0] + switchbox[1] + switchbox[2]
    signal_list.append({'ind':i,'signal':signal})
signal_list = pd.DataFrame(signal_list)

In [ ]:
switchbox = [0,0,0]
signal_list = []
for index in data.index:
    hindex = tb.get_index(index, hdata)
    tindex = tb.get_index(index, tdata)
    findex = tb.get_index(index, fdata)

    hour_sig = tb.check_bollinger(hindex, hdata, hbol)
    thirt_sig = tb.check_bollinger(tindex, tdata, tbol)
    five_sig = tb.check_bollinger(findex, fdata, fbol)

    if hour_sig == 1: switchbox[0] = 1
    elif hour_sig == -1: switchbox[0] = -1
    if five_sig == 1: switchbox[1] = 1
    elif five_sig == -1: switchbox[1] = -1
    if thirt_sig == 1: switchbox[2] = 1
    elif thirt_sig == -1: switchbox[2] = -1

    signal = switchbox[0] + switchbox[1] + switchbox[2]
    signal_list.append({'ind':index,'signal':signal})
#tb.check_bollinger(index, hdata, hbol)

In [ ]:
signals = pd.DataFrame(signal_list)
signals.set_index('ind', inplace=True)

In [ ]:
for day in tb.get_unique_dates(hdata)[5:]:
    dayta = tk.get_workday(data, day).open
    daybol = tk.get_workday(bol, day)
    plt.figure(dpi=150)
    plt.plot(dayta.index, dayta, lw=.5)
    plt.plot(daybol.index, daybol.highband, color='red', lw=.5)
    plt.plot(daybol.index, daybol.lowband, color='green', lw=.5)
    plt.title(day)
    for index in dayta.index:
        sig = signals.signal.loc[tb.get_i(index, data)]
        if sig > 0: plt.axvline(index, color='green', alpha=.25, lw=.5)
        if sig < 0: plt.axvline(index, color='red', alpha=.25, lw=.5)
    plt.show()

# Second Attempt
Ok, here goes our next attempt at making moneybot work. I suppose in a way this is v four isn't it? Oof, that hurts, but it is what it is. So how will this work. We're going to try to buy and sell every two seconds. Basically the moment we have something in our account we're going to start listing it for a small profit. We do this every second we're pretty sure things are going one way (like if we think that it's going to go down in the absolute near future). We should still be able to test this by using the interminute highs and lows.

In [ ]:
archive = tb.get_archive('cvna')
df = archive[tk.to_time(1,12):]

In [ ]:
oplo = df.low-df.open
ophi = df.high-df.open
print('oplo', oplo)
oplo.plot.line()
plt.axhline(0, color='red', lw=.5)
plt.show()
print('ophi', df.high-df.open)
ophi.plot.line()
plt.axhline(0, color='red', lw=.5)

In [ ]:
mi = oplo[oplo < 0].max()
ma = ophi[ophi > 0].min()

In [ ]:
total = len(df.index) * .01 * 1000
#for index in df.index
print(total)